In [1]:
pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('shawhin/imdb-truncated')
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

(…)-00000-of-00001-5a744bf76a1d84b2.parquet:   0%|          | 0.00/836k [00:00<?, ?B/s]

(…)-00000-of-00001-a3a52fabb70c739f.parquet:   0%|          | 0.00/853k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})


In [4]:
from transformers import MarianMTModel, MarianTokenizer

# Load pre-trained English-to-Spanish translation model
model_name = 'Helsinki-NLP/opus-mt-en-es'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Function to translate text
def translate_text(text, tokenizer, model):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Translate text
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [7]:
# Function to translate texts in batches (using CPU)
!pip install tqdm # install required package
from tqdm import tqdm # import tqdm module

def translate_batch_cpu(texts, tokenizer, model, batch_size=32):
    translated_texts = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)  # Do not move to GPU
        with torch.no_grad():
            translated = model.generate(**inputs)
        translated_texts.extend(tokenizer.batch_decode(translated, skip_special_tokens=True))
    return translated_texts

In [9]:
# Take only the first 20 rows for evaluation
subset_texts = dataset['train']['text'][:20]

# Translate the subset of data
translated_reviews_subset = translate_batch_cpu(subset_texts, tokenizer, model, batch_size=10)

# Display the results in a DataFrame
import pandas as pd

# Create a DataFrame with original and translated texts
data = {
    "Original Text": subset_texts,
    "Translated Text": translated_reviews_subset
}

df = pd.DataFrame(data)

# Show the results
print(df)


100%|██████████| 2/2 [07:54<00:00, 237.04s/it]

                                        Original Text  \
0   . . . or type on a computer keyboard, they'd p...   
1   During 1933 this film had many cuts taken from...   
2   Let me be clear. I've used IMDb for years. But...   
3   Carlos Mencia was excellent this is hour speci...   
4   I was initially dubious about this movie (mere...   
5   No matter how well meaning his "message" is - ...   
6   When you typically watch a short film your alw...   
7   How to lose friends and Alienate people came o...   
8   #1 Vampires vs. Humans<br /><br />#2 Military-...   
9   This movie is intelligent. That is, more than ...   
10  THE PERVERT'S GUIDE TO CINEMA (2007) **** <br ...   
11  Previous comments encouraged me to check this ...   
12  The wife of a stage producer in London hopes t...   
13  This had high intellectual pretensions.The mai...   
14  I had seen this movie when it got released, an...   
15  I haven't seen this film in years so my knowle...   
16  This happy-go-luck 1939 mil

In [10]:
df

,Original Text,Translated Text
0,". . . or type on a computer keyboard, they'd p...",. . . o escribir en un teclado de la computado...
1,During 1933 this film had many cuts taken from...,Durante 1933 esta película tenía muchos cortes...
2,Let me be clear. I've used IMDb for years. But...,Permíteme ser claro. He usado IMDb durante año...
3,Carlos Mencia was excellent this is hour speci...,"Carlos Mencia fue excelente, esto es una hora ..."
4,I was initially dubious about this movie (mere...,Al principio estaba dudando acerca de esta pel...
5,"No matter how well meaning his ""message"" is - ...","No importa qué tan bien significa su ""mensaje""..."
6,When you typically watch a short film your alw...,Cuando normalmente ves un cortometraje siempre...
7,How to lose friends and Alienate people came o...,¿Cómo perder amigos y Alienate gente salió en ...
8,#1 Vampires vs. Humans<br /><br />#2 Military-...,#1 Vampires vs. Humans<br /­br /·2 Escuadrón m...
9,"This movie is intelligent. That is, more than ...","Esta película es inteligente. Es decir, más qu..."


In [12]:
from nltk.translate.bleu_score import sentence_bleu

# Assuming the dataset is already split into original and translated text
original_sentences = dataset['train']['text'][:20]  # First 20 original texts
translated_sentences = translated_reviews_subset[:20]     # Corresponding translations

# Compute BLEU scores for each pair
bleu_scores = [
    sentence_bleu([original.split()], translated.split())
    for original, translated in zip(original_sentences, translated_sentences)
]

# Display results
print("BLEU Scores for First 20 Sentences:")
for i, score in enumerate(bleu_scores):
    print(f"Sentence {i+1}: BLEU Score = {score:.4f}")


BLEU Scores for First 20 Sentences:
Sentence 1: BLEU Score = 0.0438
Sentence 2: BLEU Score = 0.0327
Sentence 3: BLEU Score = 0.0000
Sentence 4: BLEU Score = 0.0000
Sentence 5: BLEU Score = 0.0000
Sentence 6: BLEU Score = 0.0000
Sentence 7: BLEU Score = 0.0000
Sentence 8: BLEU Score = 0.0570
Sentence 9: BLEU Score = 0.0216
Sentence 10: BLEU Score = 0.0000
Sentence 11: BLEU Score = 0.0265
Sentence 12: BLEU Score = 0.0000
Sentence 13: BLEU Score = 0.0181
Sentence 14: BLEU Score = 0.0000
Sentence 15: BLEU Score = 0.0000
Sentence 16: BLEU Score = 0.0000
Sentence 17: BLEU Score = 0.0670
Sentence 18: BLEU Score = 0.0000
Sentence 19: BLEU Score = 0.0000
Sentence 20: BLEU Score = 0.0000


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
pip install transformers peft


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install datasets


In [ ]:
pip install tf-keras


In [ ]:
pip install evaluate


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

C:\Users\vedas\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### dataset

In [ ]:
# # how dataset was generated

# # load imdb data
# imdb_dataset = load_dataset("imdb")

# # define subsample size
# N = 1000
# # generate indexes for random subsample
# rand_idx = np.random.randint(24999, size=N)

# # extract train and test data
# x_train = imdb_dataset['train'][rand_idx]['text']
# y_train = imdb_dataset['train'][rand_idx]['label']

# x_test = imdb_dataset['test'][rand_idx]['text']
# y_test = imdb_dataset['test'][rand_idx]['label']

# # create new dataset
# dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
#                              'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

In [ ]:
# load dataset
dataset = load_dataset('shawhin/imdb-truncated')
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [ ]:
# display % of training data with label=1
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

np.float64(0.5)

### model

In [ ]:
model_checkpoint = 'distilbert-base-uncased'
# model_checkpoint = 'roberta-base' # you can alternatively use roberta-base but this model is bigger thus training will take longer

# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


### preprocess data

In [ ]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))



In [ ]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

In [ ]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [ ]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

### Apply untrained model to text

In [ ]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Positive
Not a fan, don't recommed. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Positive


### Train model

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [ ]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [ ]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [ ]:
# define training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

C:\Users\vedas\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
pip install clearml


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["DISABLE_CLEARML_INTEGRATION"] = "true"


In [ ]:
from transformers import Trainer

# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

C:\Users\vedas\AppData\Local\Temp\ipykernel_18244\1525066263.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ClearML Task: created new task id=45563c01d5a14562b43413f729a40a66
2024-12-05 20:09:52,874 - clearml.Task - INFO - Storing jupyter notebook directly as code


ClearML results page: https://app.clear.ml/projects/f6d1583738804e54ac180b01b64f646d/experiments/45563c01d5a14562b43413f729a40a66/output/log
2024-12-05 20:09:54,714 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.477270,{'accuracy': 0.86}
2,0.437200,0.439568,{'accuracy': 0.879}
3,0.437200,0.505819,{'accuracy': 0.883}
4,0.199200,0.691299,{'accuracy': 0.884}
5,0.199200,0.943598,{'accuracy': 0.884}
6,0.020900,1.000411,{'accuracy': 0.884}
7,0.020900,1.016043,{'accuracy': 0.895}
8,0.022200,1.123469,{'accuracy': 0.885}
9,0.022200,1.045221,{'accuracy': 0.897}
10,0.014900,1.048401,{'accuracy': 0.893}


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Trainer is attempting to log a value of "{'accuracy': 0.86}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.86}" of type <class 'dict'> for key "eval_accuracy" as a scalar. This invocation of ClearML logger's  report_scalar() is incorrect so we dropped this attribute.


...

2024-12-05 22:04:48,718 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/HuggingFace%20Transformers/Trainer.45563c01d5a14562b43413f729a40a66/models/checkpoint-2500.zip


Trainer is attempting to log a value of "{'accuracy': 0.893}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.893}" of type <class 'dict'> for key "eval_accuracy" as a scalar. This invocation of ClearML logger's  report_scalar() is incorrect so we dropped this attribute.


2024-12-05 22:08:40,217 - clearml.storage - INFO - Starting upload: C:\Users\vedas\AppData\Local\Temp\model_package.vqg954qp.zip => https://files.clear.ml/HuggingFace Transformers/Trainer.45563c01d5a14562b43413f729a40a66/models/checkpoint-2500.zip


████████████████████████████████▉ 100% | 8.13/8.14 MB [00:02<00:00,  4.03MB/s]: 


TrainOutput(global_step=2500, training_loss=0.13888909111022948, metrics={'train_runtime': 7131.0027, 'train_samples_per_second': 1.402, 'train_steps_per_second': 0.351, 'total_flos': 1112883852759936.0, 'train_loss': 0.13888909111022948, 'epoch': 10.0})

In [ ]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Positive
